# NVT

## Limitations

There is no support for `define = -DPOSRES` that is present in the .mdp files if we would want to add restrictions to specific atoms we would have to do it manually

In [ ]:
import openmm as mm
import openmm.app as app
import openmm.unit as unit
from sys import stdout

# Load your system and topology
# Replace 'your_pdb_file.pdb' and 'your_forcefield.xml' with your actual files
pdb = app.PDBFile('input.pdb')
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')


modeller = app.Modeller(pdb.topology, pdb.positions)
modeller.deleteWater()
residues=modeller.addHydrogens(forcefield)

modeller.addSolvent(forcefield, padding=1.0*unit.nanometer)


# Create the system
system = forcefield.createSystem(modeller.topology, 
                                 nonbondedMethod=app.PME,
                                 nonbondedCutoff=1.2*unit.nanometers, 
                                 constraints=app.HBonds, 
                                 ewaldErrorTolerance=0.0005)

# Integrator settings
integrator = mm.LangevinIntegrator(300*unit.kelvin,    # ref_t
                                   1/unit.picosecond,  # tau_t
                                   0.002*unit.picoseconds)  # dt

# Periodic boundary conditions
pdb.topology.setPeriodicBoxVectors(system.getDefaultPeriodicBoxVectors())

# Create the simulation object
simulation = app.Simulation(modeller.topology, system, integrator)

# Set the initial positions
simulation.context.setPositions(modeller.positions)

# Minimize energy
simulation.minimizeEnergy()

# Set velocities to match the given temperature
simulation.context.setVelocitiesToTemperature(300*unit.kelvin, -1)

# Set up reporters to output data
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 2500))  # nstxout-compressed
simulation.reporters.append(app.StateDataReporter(stdout, 50,  # nstlog
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True, 
                                                  volume=True))
simulation.reporters.append(app.StateDataReporter('energy.log', 500,  # nstenergy
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True))
simulation.reporters.append(app.CheckpointReporter("nvt.cpt", 1000))


# Run the simulation
simulation.step(50000)  # nsteps

# Save final state
simulation.saveCheckpoint("nvt.cpt")



# NPT

In [ ]:


system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
simulation.context.reinitialize(preserveState=True)

# Set up reporters to output data
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 2500))  # nstxout-compressed
simulation.reporters.append(app.StateDataReporter(stdout, 50,  # nstlog
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True, 
                                                  volume=True))
simulation.reporters.append(app.StateDataReporter('energy.log', 500,  # nstenergy
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True))
simulation.reporters.append(app.CheckpointReporter("md.cpt", 1000))


# Run the simulation
simulation.step(50000)  # nsteps
simulation.saveCheckpoint("npt.cpt")


print("Simulation complete.")

# MD

In [ ]:




# Set up reporters to output data
simulation.reporters.append(app.DCDReporter('trajectory.dcd', 2500))  # nstxout-compressed
simulation.reporters.append(app.StateDataReporter(stdout, 50,  # nstlog
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True, 
                                                  volume=True))
simulation.reporters.append(app.StateDataReporter('energy.log', 500,  # nstenergy
                                                  step=True, 
                                                  potentialEnergy=True, 
                                                  kineticEnergy=True, 
                                                  totalEnergy=True, 
                                                  temperature=True))
simulation.reporters.append(app.CheckpointReporter("md.cpt", 1000))


# Run the simulation
simulation.step(500000)  # nsteps
simulation.saveCheckpoint("md.cpt")


print("Simulation complete.")

In [ ]:
=